# Age of contracts

Something that will be useful later will be the average age of contracts

In [1]:
from datetime import date, datetime, timedelta
from copy import copy
import numpy as np
import pandas as pd
import sqlalchemy as sa
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import Scatter, Figure, Layout, Histogram, Heatmap
from plotly import tools
init_notebook_mode(connected=True)

# Custom functions for data exchange
from utils.data_utils import update_data, df_from_table, gen_datetime_col

# Custom plotting
from utils.plot_utils import plot, plot_subplot

# Create a database file using sqlite through sqlalchemy
engine = sa.create_engine('sqlite:///cl_basic_data_analysis.db')

# Grab our data again. 
df_query = df_from_table('cl_data', engine, 'index')
df_query.head()

,date,open,high,low,close,volume,openint,contract_name,year,month,day,contract_symbol,contract_year,contract_month
index,,,,,,,,,,,,,,
0,1983-03-30,29.01,29.56,29.01,29.40,949,470,CL1983-06,1983,3,30,CL,1983,6
1,1983-03-31,29.40,29.60,29.25,29.29,521,523,CL1983-06,1983,3,31,CL,1983,6
2,1983-04-04,29.30,29.70,29.29,29.44,156,583,CL1983-06,1983,4,4,CL,1983,6
3,1983-04-05,29.50,29.80,29.50,29.71,175,623,CL1983-06,1983,4,5,CL,1983,6
4,1983-04-06,29.90,29.92,29.65,29.90,392,640,CL1983-06,1983,4,6,CL,1983,6


In [46]:
g = df_query[['contract_name', 'date']].groupby('contract_name')
ages = (g.max()-g.min())
ages['date'] = ages['date'].map(lambda x: x.days)
ages.rename(columns={'date': 'age'}, inplace=True)
ages['year'] = ages.index.map(lambda x: int(x[2:6]))
# Filter future years
ages = ages[ages['year'] < 2017]

In [47]:
age_by_year = ages.groupby('year').mean()
data=[Scatter(x=age_by_year.index, y=age_by_year.age)]
plot(data, title='Average age of contracts by year', show_legend=False)

In [48]:
age_by_year.tail()

,age
year,
2012,2054.333333
2013,2061.916667
2014,2120.166667
2015,2179.083333
2016,2205.666667


In [72]:
ages['month'] = ages.index.map(lambda x: int(x[-2:]))
ages['date'] = [pd.Timestamp(y, m, 1) for y, m in zip(ages['year'].values, ages['month'].values)]
g = ages.set_index('date').groupby(pd.Grouper(freq='M'))
age_by_month = g.mean()
age_by_month_limited = age_by_month[age_by_month['year'] >= 2012]
data=[Scatter(x=age_by_month_limited.index, y=age_by_month_limited.age)]
plot(data, title='Average age of contracts by month between 2012 and 2017', show_legend=False)